## DAIS 2025 DQX Demo Session 
This notebook illustrates example usage of DQX for a fictional Manufacturing Company "Machina Metrics". <br>
Watch **DAIS Demo Session Recording** that showcases this demo: https://www.youtube.com/watch?v=e5Qvx_gnxTE


<h1>
MachinaMetrics - A Manufacturing Company
</h1>

### Aspirations - Proactive Machine Maintenance

MachinaMetrics is a leading-edge manufacturing company whose CTO is spearheading the adoption of an AI-driven predictive maintenance solution. By harnessing real-time machine status data and historical maintenance schedules, this technology will proactively forecast service needs, minimize unplanned downtime, and optimize maintenance cycles. The result is a significant reduction in operational costs, extended equipment lifespan, and maximized production efficiency-positioning MachinaMetrics as an industry innovator in smart manufacturing.

# DQX - The Data Quality Framework

### DQX options


| Type | Description | Features|
| ----------- | ----------- | ----------- |
| Installation| Deploy as a Library | Deploy as workspace tool 
| Usage | Use in Spark Dataframes| Use with DLT|
| Quality Rules| Define as YAML| Define as Code|

### DQX in Lakehouse Architecture


![dqx-in-lake.png](./images/dqx-in-lake.png "dqx-in-lake.png")

### DQX - Install as Library <br>


In [0]:
%pip install databricks-labs-dqx

In [0]:
dbutils.library.restartPython()

In [0]:
import os
os.getcwd()

In [0]:
import os
workspace_root_path = os.getcwd()
quality_rules_path = f"{workspace_root_path}/quality_rules"

### Sample Data Generation



In [0]:
default_database = "main"
default_schema_name = "default"

dbutils.widgets.text("demo_database", default_database, "Catalog Name")
dbutils.widgets.text("demo_schema", default_schema_name, "Schema Name")

database = dbutils.widgets.get("demo_database")
schema = dbutils.widgets.get("demo_schema")

print(f"Selected Catalog for Demo Dataset: {database}")
print(f"Selected Schema for Demo Dataset: {schema}")

spark.sql(f"CREATE CATALOG IF NOT EXISTS {database}")
spark.sql(f"USE CATALOG {database}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")
spark.sql(f"USE SCHEMA {schema}")

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
import delta

sensor_table = f"{database}.{schema}.sensor_data"

if spark.catalog.tableExists(sensor_table) and spark.table(sensor_table).count() > 0:
    print(
        f"Table {sensor_table} already exists with demo data. Skipping data generation"
    )
else:
    # 1. Enhanced Sensor Data with ingest_date and multiple rows per ingest_date
    sensor_schema = StructType(
        [
            StructField("sensor_id", StringType(), False),
            StructField("machine_id", StringType(), True),  # Allow null values
            StructField("sensor_type", StringType(), False),
            StructField("reading_value", DoubleType(), True),
            StructField("reading_timestamp", TimestampType(), True),
            StructField("calibration_date", DateType(), True),
            StructField("battery_level", IntegerType(), True),
            StructField("facility_zone", StringType(), True),
            StructField("is_active", BooleanType(), True),
            StructField("firmware_version", StringType(), True),
            StructField("ingest_date", DateType(), True),
        ]
    )

    sensor_data = [
        # Ingest date 2025-04-28
        (
            "SEN-001",
            "MCH-001",
            "temperature",
            72.4,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        (
            "SEN-002",
            "MCH-001",
            "pressure",
            2.1,
            datetime.strptime("2025-04-28 14:32:05", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-03-15", "%Y-%m-%d").date(),
            45,
            "Zone-A",
            True,
            "v1.9.4",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        (
            "SEN-003",
            "MCH-002",
            "vibration",
            0.02,
            datetime.strptime("2025-04-28 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            92,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),  # Invalid calibration
        # Ingest date 2025-04-29
        (
            "SEN-001",
            "MCH-001",
            "temperature",
            73.5,
            datetime.strptime("2025-04-29 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            80,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),
        (
            "SEN-002",
            "MCH-001",
            "pressure",
            2.3,
            datetime.strptime("2025-04-29 14:32:05", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-03-15", "%Y-%m-%d").date(),
            50,
            "Zone-A",
            True,
            "v1.9.4",
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),
        (
            "SEN-004",
            None,
            "temperature",
            74.5,
            datetime.strptime("2025-04-29 14:32:15", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-15", "%Y-%m-%d").date(),
            10,
            "Zone-C",
            True,
            "invalid_ver",
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),  # Multiple issues
        # Ingest date 2025-04-30
        (
            "SEN-001",
            "MCH-001",
            "temperature",
            74.0,
            datetime.strptime("2025-04-30 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            75,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
        (
            "SEN-003",
            "MCH-002",
            "vibration",
            0.03,
            datetime.strptime("2025-04-30 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            90,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
        # Bad Data Insertion
        # Sensor is empty
        (
            "",
            "MCH-002",
            "vibration",
            0.03,
            datetime.strptime("2025-04-30 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            90,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
        # Machine_id is empty
        (
            "SEN-001",
            "",
            "temperature",
            72.4,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        # Invalid Temperature
        (
            "SEN-001",
            "MCH-001",
            "temperature",
            735,
            datetime.strptime("2025-04-29 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            80,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),
        # Sensor regex pattern is wrong
        (
            "SEN002",
            "MCH-002",
            "vibration",
            0.03,
            datetime.strptime("2025-04-30 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            90,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
        (
            "SEN001",
            "",
            "temperature",
            72.4,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        # Reading TS in future
        (
            "SEN-001",
            "",
            "temperature",
            72.4,
            datetime.strptime("2026-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        (
            "SEN-002",
            "MCH-001",
            "temperature",
            735,
            datetime.strptime("2026-04-29 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            80,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),
        # Invalid Temperature
        (
            "SEN-003",
            "MCH-001",
            "vibration",
            0.03,
            datetime.strptime("2025-04-30 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            90,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-05-01", "%Y-%m-%d").date(),
        ),
        (
            "SEN-004",
            "MCH-001",
            "temperature",
            15000.0,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        # Invalid wrong sensor pattern and ts in future
        (
            "SE003",
            "MCH-001",
            "vibration",
            0.03,
            datetime.strptime("2026-04-30 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            90,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-05-01", "%Y-%m-%d").date(),
        ),
        # Invalid Temperature and wrong sensor pattern
        (
            "SEN004",
            "MCH-001",
            "temperature",
            724,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        # Invalid Firmware Version and wrong sensor pattern
        (
            "SEN004",
            "MCH-001",
            "temperature",
            724,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "b2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        # Machine ID regex pattern is wrong
        (
            "SEN-002",
            "MCH2",
            "vibration",
            0.03,
            datetime.strptime("2025-04-30 14:32:10", "%Y-%m-%d %H:%M:%S"),
            None,
            90,
            "Zone-B",
            False,
            "v3.0.0",
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
        (
            "",
            "MCH2",
            "temperature",
            72.4,
            datetime.strptime("2025-04-28 14:32:00", "%Y-%m-%d %H:%M:%S"),
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            85,
            "Zone-A",
            True,
            "v2.3.1",
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
    ]

    sensor_df = spark.createDataFrame(sensor_data, schema=sensor_schema)
    sensor_df.write.mode("overwrite").saveAsTable(sensor_table)

In [0]:
from decimal import Decimal

maintenance_table = f"{database}.{schema}.maintenance_data"

if (
    spark.catalog.tableExists(maintenance_table)
    and spark.table(maintenance_table).count() > 0
):
    print(
        f"Table {maintenance_table} already exists with demo data. Skipping data generation"
    )
else:
    # 2. Enhanced Maintenance Data with ingest_date and multiple rows per ingest_date
    maintenance_schema = StructType(
        [
            StructField("maintenance_id", StringType(), False),
            StructField("machine_id", StringType(), False),
            StructField("maintenance_type", StringType(), True),
            StructField("maintenance_date", DateType(), True),
            StructField("duration_minutes", IntegerType(), True),
            StructField("cost", DecimalType(10, 2), True),
            StructField("next_scheduled_date", DateType(), True),
            StructField("work_order_id", StringType(), True),
            StructField("safety_check_passed", BooleanType(), True),
            StructField("parts_list", ArrayType(StringType()), True),
            StructField("ingest_date", DateType(), True),
        ]
    )

    maintenance_data = [
        # Ingest date 2025-04-28
        (
            "MTN-001",
            "MCH-001",
            "preventive",
            datetime.strptime("2025-04-01", "%Y-%m-%d").date(),
            120,
            Decimal("450.00"),
            datetime.strptime("2025-07-01", "%Y-%m-%d").date(),
            "WO-001",
            True,
            ["filter", "gasket"],
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),
        (
            "MTN-002",
            "MCH-002",
            "corrective",
            datetime.strptime("2025-04-15", "%Y-%m-%d").date(),
            240,
            Decimal("1200.50"),
            datetime.strptime("2026-04-01", "%Y-%m-%d").date(),
            "WO-002",
            False,
            ["motor"],
            datetime.strptime("2025-04-28", "%Y-%m-%d").date(),
        ),  # Future date issue
        # Ingest date 2025-04-29
        (
            "MTN-003",
            "MCH-003",
            None,
            datetime.strptime("2025-04-20", "%Y-%m-%d").date(),
            -30,
            Decimal("-500.00"),
            datetime.strptime("2024-04-20", "%Y-%m-%d").date(),
            "INVALID",
            None,
            [],
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),  # Multiple issues
        (
            "MTN-004",
            "MCH-001",
            "predictive",
            datetime.strptime("2025-04-25", "%Y-%m-%d").date(),
            180,
            Decimal("800.00"),
            datetime.strptime("2025-10-01", "%Y-%m-%d").date(),
            "WO-003",
            True,
            ["sensor"],
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
        ),
        # Ingest date 2025-04-30
        (
            "MTN-005",
            "MCH-002",
            "preventive",
            datetime.strptime("2025-04-29", "%Y-%m-%d").date(),
            90,
            Decimal("300.00"),
            datetime.strptime("2025-07-15", "%Y-%m-%d").date(),
            "WO-004",
            True,
            ["valve"],
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
        (
            "MTN-006",
            "MCH-003",
            "corrective",
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
            60,
            Decimal("150.00"),
            datetime.strptime("2025-08-01", "%Y-%m-%d").date(),
            "WO-005",
            False,
            ["pump"],
            datetime.strptime("2025-04-30", "%Y-%m-%d").date(),
        ),
    ]

    maintenance_df = spark.createDataFrame(maintenance_data, schema=maintenance_schema)
    maintenance_df.write.mode("overwrite").saveAsTable(maintenance_table)

### Understanding the datasets



 
### Machine Sensor Readings Dataset

| Column Name        | Data Type    | Description                                      | Example Value         |
|--------------------|-------------|--------------------------------------------------|----------------------|
| `sensor_id`        | string      | Unique sensor identifier                         | SEN-001              |
| `machine_id`       | string      | Linked machine identifier                        | MCH-001              |
| `sensor_type`      | string      | Type of sensor (temperature, pressure, etc.)     | temperature          |
| `reading_value`    | double      | Value recorded by the sensor                     | 72.4                 |
| `reading_timestamp`| timestamp   | Time the reading was taken                       | 2025-04-28 14:32:00  |
| `calibration_date` | date        | Last calibration date of the sensor              | 2025-04-01           |
| `battery_level`    | int         | Battery percentage (0-100)                       | 85                   |
| `facility_zone`    | string      | Plant zone or location                           | Zone-A               |
| `is_active`        | boolean     | Whether the sensor is active                     | true                 |
| `firmware_version` | string      | Sensor firmware version                          | v2.3.1               |
| `ingest_date`      | date        | Date the record was ingested                     | 2025-04-28           |



### Some sample "Sensor Table" Data

In [0]:

sensor_bronze_df = spark.read.table(sensor_table)
print("=== Sensor Data Sample ===")
display(sensor_bronze_df.limit(10))



### Maintenance Records Dataset

| Column Name           | Data Type        | Description                                   | Example Value           |
|-----------------------|-----------------|-----------------------------------------------|------------------------|
| `maintenance_id`      | string          | Unique maintenance event identifier           | MTN-001                |
| `machine_id`          | string          | Linked machine identifier                     | MCH-001                |
| `maintenance_type`    | string          | Type of maintenance (preventive, corrective)  | preventive             |
| `maintenance_date`    | date            | Date maintenance was performed                | 2025-04-01             |
| `duration_minutes`    | int             | Duration of maintenance in minutes            | 120                    |
| `cost`                | decimal(10,2)   | Cost of maintenance                           | 450.00                 |
| `next_scheduled_date` | date            | Next scheduled maintenance date               | 2025-07-01             |
| `work_order_id`       | string          | Associated work order identifier              | WO-001                 |
| `safety_check_passed` | boolean         | Whether safety check was passed               | true                   |
| `parts_list`          | array   | List of parts replaced or serviced            | ["filter", "gasket"]   |
| `ingest_date`         | date            | Date the record was ingested                  | 2025-04-28             |



### Some sample "Maintenance Table" Data

In [0]:
mntnc_bronze_df = spark.read.table(maintenance_table)
print("=== Maintenance Data Sample ===")
display(mntnc_bronze_df.limit(10))


### Problem - Team doesn't know the Quality Rules for Maintenance Dataset
### Feature - Infer the Data Quality Rules using DQX


**Step-1**. Read Raw Data and Instantiate DQX 

In [0]:
import os
from databricks.labs.dqx.profiler.profiler import DQProfiler
from databricks.labs.dqx.profiler.generator import DQGenerator
from databricks.labs.dqx.engine import DQEngine
from databricks.sdk import WorkspaceClient
from pprint import pprint

# Read Input Data
mntnc_bronze_df = spark.read.table(maintenance_table)

# Instantiate DQX engine
ws = WorkspaceClient()
dq_engine = DQEngine(ws)


**Step-2**. Run DQX Profiler and **Infer** Quality Rules

In [0]:
# Profile Inpute Data
profiler = DQProfiler(ws)
summary_stats, profiles = profiler.profile(mntnc_bronze_df)

# Generate DQX quality rules/checks
generator = DQGenerator(ws)
checks = generator.generate_dq_rules(profiles)  # with default level "error"

In [0]:
print("=== Inferred DQ Checks ===")

for idx, check in enumerate(checks):
   print(f"\n========Check {idx} ==========")
   pprint(check)

In [0]:
# save checks in arbitrary workspace location
maintenance_quality_rules = f"{quality_rules_path}/maintenance_dq_rules.yml"
dq_engine.save_checks_in_workspace_file(checks, workspace_path=maintenance_quality_rules)

# display the link to the saved checks
displayHTML(f'<a href="/#workspace{maintenance_quality_rules}" target="_blank">Maintenance Data Quality Rules YAML</a>')

In [0]:
# or save in delta table
fq_tbl = f"{database}.{schema}.maintenance_inferred_quality_rules"
dq_engine.save_checks_in_table(table_name=fq_tbl, checks=checks, run_config_name="maintenance")


**Step-3**. Apply Inferred Quality Rules to Input Data

In [0]:
# Load checks from workspace file
quality_checks = dq_engine.load_checks_from_workspace_file(workspace_path=maintenance_quality_rules)
# or Load checks from a table
# quality_checks = dq_engine.load_checks_from_table(table_name=fq_tbl, run_config_name="maintenance")

# Apply checks on input data
valid_df, quarantined_df = dq_engine.apply_checks_by_metadata_and_split(mntnc_bronze_df, quality_checks)

print("=== Maintenance Bad Data Sample ===")
display(quarantined_df)


### Data Quality Rules for Machine Sensor Data

| Rule Type             | Example Rule                                                                                           | Purpose / Impact                                                        |DQ Rule|Quality Error Level|
|-----------------------|-------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|-|--|
| **Completeness**      | Required fields (`sensor_id`, `machine_id`) must not be null    | Ensures all critical data is present and usable     |`is_not_null_and_not_empty`                    |ERROR|
| **Range / Domain**    | `reading_value` (temperature): 0–100 | Detects outliers and sensor faults; ensures physical plausibility       |**FILTER quality Check + `is_in_range`**| WARN|
| **Format Standardization**  |  `machine_id` follows standard format                             | Standardizes data for integration and analysis                          |`regex_match` |WARN|
| **Timeliness**        | `reading_timestamp` is not in the future; beyond 3 days                                     | Prevents erroneous time-series data                            |`is_not_in_future` |ERROR|
| **Correctness**        | `calibration_date` is eariler than `reading_timestamp`| Prevents erroneous sesnor readings data                            |`SQL Expression` |ERROR|





### Feature: Quality Rules as YAML 
Read quality rules from the a YAML file and apply checks on a Dataframe


In [0]:
sensor_bronze_df = spark.read.table(sensor_table)
sensor_quality_checks = dq_engine.load_checks_from_workspace_file(workspace_path=f"{quality_rules_path}/sensor_dq_rules.yml")

### Features: Quarantine Bad Data & Perform Granular Issue Detection

In [0]:
valid_df, quarantined_df = dq_engine.apply_checks_by_metadata_and_split(sensor_bronze_df, sensor_quality_checks)

print("=== Bad Data DF ===")
display(quarantined_df)

### Feature: Bring / Build Your own Rule (Check)
|Dataset| Rule Type             | Example Rule                                                                                           | Purpose / Impact                                                        |DQ Rule|
|-|-----------------------|-------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|--|
|Sensor Data| **Standardization**          | `firmware_version` starts with "v"  | Ensures firmware version value is a standard value | Custom Rule Development| 


In [0]:
import pyspark.sql.functions as F
from pyspark.sql import Column as col
from databricks.labs.dqx.check_funcs import make_condition

def firmware_version_start_with_v(column: str) -> col:
    column_expr = F.expr(column)
    
    quality_rule_expr = ~(column_expr.startswith("v"))
    quality_rule_err_msg = f"firmware_version doesn't starts with 'v'"
    quality_rule_err_col_name = f"firmware_version_not_starts_with_v"

    return make_condition(quality_rule_expr, quality_rule_err_msg, quality_rule_err_col_name)


### Feature : YAML Way

[Sensor Data Quality Custom Rules YAML](https://e2-demo-field-eng.cloud.databricks.com/editor/files/2408509664666985?o=1444828305810485)



**Step-1** Read the input data and instantiate DQX Engine

In [0]:
sensor_bronze_df = spark.read.table(sensor_table)
sensor_quality_checks = dq_engine.load_checks_from_workspace_file(
    workspace_path=f"{quality_rules_path}/sensor_dq_rules_custom.yml")

dq_engine = DQEngine(WorkspaceClient())

In [0]:
# Define the custom check 
custom_check_functions = {"firmware_version_start_with_v": firmware_version_start_with_v}  # list of custom check functions

# Apply the custom check on the bronze data
valid_df, quarantined_df = dq_engine.apply_checks_by_metadata_and_split(sensor_bronze_df, sensor_quality_checks, custom_check_functions)
display(quarantined_df)

sensor_quarantine_table = f"{database}.{schema}.sensor_quarantine"
quarantined_df.write.mode("overwrite").saveAsTable(sensor_quarantine_table)

### Feature: Visualize Quality on Pre-Configured Dashboard


https://e2-demo-field-eng.cloud.databricks.com/dashboardsv3/01f025eecf261a81996b8991df9e870b/published?o=1444828305810485

![dashboard.png](./images/dashboard.png "dashboard.png")
